In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
# load data
data_path = '../data/augmented_and_padded_data.npz'
data = np.load(data_path)
melody_pcps = data['melody_pcps']
chord_pcps = data['chord_pcps']
print('melody_pcps.shape: ', melody_pcps.shape)
print('chord_pcps.shape: ', chord_pcps.shape)

melody_pcps.shape:  (5328, 129, 12)
chord_pcps.shape:  (5328, 129, 12)


In [15]:
np.nonzero(melody_pcps[0,0,:])[0] + 6

array([ 6, 15])

In [21]:
class BinarySerializer:
    def __init__(self, pad_to_length=0):
        '''
        0: padding
        1: start_melody
        2: melody segment seperator
        3: chord segment separator
        4: start harmonizing
        5: end harmonizing
        6 to 6+11=17: melody pitch class
        18 to 18+11=29: chord pitch class
        
        vocab_size = 30
        '''
        self.padding = 0
        self.start_melody = 1
        self.melody_segment_separator = 2
        self.chord_segment_separator = 3
        self.start_harmonizing = 4
        self.end_harmonizing = 5
        self.melody_offset = 6
        self.chord_offset = 18
        self.max_seq_length = 0
        self.pad_to_length = pad_to_length
        self.vocab_size = 30
    # end init
    def sequence_serialization(self, melody, chords):
        t = []
        t.append(self.start_melody)
        for i in range(melody.shape[0]):
            # check if melody pcs exist
            m = melody[i,:]
            nzm = np.nonzero(m)[0]
            if len(nzm) > 0:
                t.append( self.melody_segment_separator )
                t.extend( nzm + self.melody_offset )
        t.append(self.start_harmonizing)
        for i in range(chords.shape[0]):
            # check if chord pcs exist
            c = chords[i,:]
            nzc = np.nonzero(c)[0]
            if len(nzc) > 0:
                t.append( self.chord_segment_separator )
                t.extend( nzc + self.chord_offset )
        t.append(self.end_harmonizing)
        if len(t) > self.max_seq_length:
            self.max_seq_length = len(t)
        while self.pad_to_length > len(t):
            t.append(self.padding)
        return t
    # end sequence_serialization
# end class BinarySerializer

In [22]:
binser = BinarySerializer()
s = binser.sequence_serialization( melody_pcps[0,:,:], chord_pcps[0,:,:] )
print(s)

[1, 2, 6, 15, 2, 17, 2, 6, 15, 2, 17, 2, 13, 17, 2, 15, 2, 13, 17, 2, 15, 2, 6, 15, 2, 17, 2, 6, 15, 2, 13, 17, 2, 10, 2, 10, 2, 7, 10, 2, 9, 2, 7, 10, 2, 9, 2, 9, 17, 2, 7, 2, 9, 17, 2, 7, 2, 7, 10, 2, 9, 2, 7, 10, 2, 9, 17, 2, 14, 2, 13, 2, 6, 15, 2, 17, 2, 6, 15, 2, 17, 2, 13, 17, 2, 15, 2, 13, 17, 2, 15, 2, 6, 15, 2, 17, 2, 6, 15, 2, 13, 17, 2, 13, 2, 13, 2, 15, 2, 13, 2, 11, 2, 10, 2, 13, 2, 11, 2, 8, 10, 2, 6, 15, 2, 17, 2, 6, 15, 2, 10, 17, 2, 6, 2, 6, 4, 3, 18, 20, 23, 27, 3, 20, 23, 25, 29, 3, 18, 20, 23, 27, 3, 20, 23, 25, 29, 3, 18, 22, 25, 29, 3, 18, 21, 23, 27, 3, 20, 22, 25, 29, 3, 19, 22, 25, 27, 3, 18, 20, 23, 27, 3, 20, 23, 25, 29, 3, 18, 20, 23, 27, 3, 20, 23, 25, 29, 3, 18, 22, 25, 29, 3, 18, 22, 25, 29, 3, 19, 22, 24, 27, 3, 21, 24, 27, 29, 3, 19, 22, 24, 27, 3, 21, 24, 27, 29, 3, 21, 22, 26, 29, 3, 19, 22, 25, 27, 3, 21, 24, 26, 29, 3, 19, 23, 26, 29, 3, 19, 22, 24, 27, 3, 21, 24, 27, 29, 3, 19, 22, 24, 27, 3, 21, 24, 27, 29, 3, 21, 22, 26, 29, 3, 20, 23, 25, 29, 3

In [23]:
binser = BinarySerializer()
serialized_data = []
for i in tqdm(range(melody_pcps.shape[0])):
    serialized_data.append( binser.sequence_serialization( melody_pcps[i,:,:], chord_pcps[i,:,:] ) )

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5328/5328 [00:01<00:00, 3625.90it/s]


In [24]:
print(binser.max_seq_length)
print(len(serialized_data))

1063
5328


In [29]:
m = 0
# print(serialized_data[0])
for s in serialized_data:
    if np.max(s) > m:
        m = np.max(s)
print(m)

29


In [30]:
import pickle

with open('serializer.pkl', 'wb') as outp:
    pickle.dump(binser, outp, pickle.HIGHEST_PROTOCOL)